<a href="https://colab.research.google.com/github/nilobarrantes/Projetos_Deep_Learning/blob/master/Reconhecer_Cavalo_Humano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

# **Cavalos vs Humanos**


![](https://drive.google.com/uc?export=view&id=1a7xwLCdqovYxoJq3M4CM8v6cZX-k16KT)

O conjunto de dados Cavalos vs. Humanos é um conjunto de dados de visão computacional padrão que envolve a classificação de fotos como contendo um cavalo ou um humano.

Embora o problema pareça simples, ele só foi abordado de forma eficaz nos últimos anos usando redes neurais convolucionais (CNN) de aprendizado profundo.

### **Como funcionam as CNNs?**
As redes convolucionais funcionam transformando os pixels de uma imagem em informações que podem ser entendidas pela máquina. As etapas envolvidas no reconhecimento de imagem são:



* Convolution
* Pooling
* Flattening
* Fully connected layer



## **Importando Imagens de Treinamento e validaçao.**
Usaremos um conjunto de imagens (treinamento) para ``ensinar'' à rede neural a diferença entre um cavalo e um humano. Ou seja, o conjunto de treinamento são os dados usados para informar ao modelo de rede neural que 'é assim que se parece um cavalo', 'é assim que se parece um humano' etc.

Outro conjunto de imagens (validação), será usado para testar se nossa rede, já treinada, consegue distinguir um cavalo de um humano. 


In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
    -O /tmp/validation-horse-or-human.zip

O código python a seguir usará a biblioteca OS para usar as bibliotecas do sistema operacional, fornecendo acesso ao sistema de arquivos e a biblioteca zipfile, permitindo a descompactação dos dados.

In [ ]:
import os
import zipfile

local_zip = '/tmp/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/horse-or-human')
local_zip = '/tmp/validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/validation-horse-or-human')
zip_ref.close()

O conteúdo do .zip é extraído para o diretório base `/ tmp / horse-or-human`, que por sua vez contém subdiretórios` horses` e `humanos`.

Vamos definir cada um desses diretórios:

In [ ]:
# Diretório de treinamento com as fotos de cavalos
train_horse_dir = os.path.join('/tmp/horse-or-human/horses')

# Diretório de treinamento com as fotos de humanos 
train_human_dir = os.path.join('/tmp/horse-or-human/humans')

# Diretório de validação com as fotos de cavalos
validation_horse_dir = os.path.join('/tmp/validation-horse-or-human/horses')

# Diretório de validação com as fotos de humanos
validation_human_dir = os.path.join('/tmp/validation-horse-or-human/humans')

In [ ]:
train_horse_names = os.listdir(train_horse_dir)
train_human_names = os.listdir(train_human_dir)

validation_horse_hames = os.listdir(validation_horse_dir)
validation_human_names = os.listdir(validation_human_dir)

## Construindo um pequeno modelo do zero

Mas antes de continuar, vamos começar a definir o modelo:

A etapa 1 será importar a biblioteca TensorFlow.

**TensorFlow** é uma biblioteca de código aberto compatível com Python para computaçao numérica que torna o *,achine learning* mais rápido e fácil.

In [ ]:
import tensorflow as tf

Em seguida, adicionamos camadas convolucionais e achatamos(flatten) o resultado final para alimentar as camadas densamente conectadas.

Finalmente, adicionamos as camadas densamente conectadas.

Observe que, como estamos enfrentando um problema de classificação de duas classes, ou seja, um *problema de classificação binária*, finalizaremos  nossa rede com uma [ativação sigmoide](https://wikipedia.org/wiki/Sigmoid_function), de modo que a saída de nossa rede será um único escalar entre 0 e 1, codificando a probabilidade de que a imagem atual seja da classe 1 (em oposição à classe 0).

In [ ]:
model = tf.keras.models.Sequential([
    # Observe que p input_shape é o tamanho desejado da imagem 150x150 com 3 bytes de cor
    # Esta é a primeira convolução
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # A segunda convolução
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # A terceira convolução
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # A quarta convolução
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # A quinta convolução
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Achate os resultados para alimentar um DNN
    tf.keras.layers.Flatten(),
    # camada oculta de 512 neurônios
    tf.keras.layers.Dense(512, activation='relu'),
    # Apenas 1 neurônio de saída. Ele conterá um valor
    # de 0-1 onde 0 para uma classe ('cavalos') e 1 para a outra ('humanos')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

A chamada do método model.summary () imprime um resumo da rede neural.

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 15, 15, 64)       


A coluna "formato de saída" mostra como o tamanho do seu mapa de carateristicas evolui em cada camada sucessiva. As camadas de **convolução** reduzem um pouco o tamanho dos mapas de carateristicas devido ao preenchimento, e cada camada de agrupamento (**pooling**) divide as dimensões pela metade.

A seguir, vamos configurar as especificações para o treinamento do modelo. Vamos treinar nosso modelo com a perda `binary_crossentropy`, porque é um problema de classificação binária e nossa ativação final é um sigmóide. Nós usaremos o otimizador `rmsprop` com uma taxa de aprendizagem de` 0,001`. Durante o treinamento, vamos monitorar a precisão da classificação.

**NOTA**: Neste caso, usar o [algoritmo de otimização RMSprop](https://wikipedia.org/wiki/Stochastic_gradient_descent#RMSProp) é preferível a [descida gradiente estocástico](https://developers.google.com/machine-learning/glossary/#SGD) (SGD), porque o RMSprop automatiza o ajuste da taxa de aprendizagem para nós. (Outros otimizadores, como [Adam](https://wikipedia.org/wiki/Stochastic_gradient_descent#Adam) e [Adagrad](https://developers.google.com/machine-learning/glossary/#AdaGrad), também adaptarao automaticamente a taxa de aprendizagem durante o treinamento e funcionariam igualmente bem.)

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

### Pré-processamento de dados

Vamos configurar geradores de dados que irão ler imagens em nossas pastas de origem, convertê-los em tensores `float32` e alimentá-los (com seus rótulos) para nossa rede. Teremos um gerador para as imagens de treinamento e outro para as imagens de validação. Nossos geradores produzirão lotes de imagens de tamanho 300x300 e seus rótulos (binários).

Os dados que vão para as redes neurais geralmente devem ser normalizados para torná-los mais receptivos ao processamento pela rede. Em nosso caso, iremos pré-processar nossas imagens normalizando os valores de pixel para que fiquem no intervalo `[0, 1]` (originalmente todos os valores estão no intervalo `[0, 255]` intervalo).

No Keras, isso pode ser feito por meio da classe `keras.preprocessing.image.ImageDataGenerator` usando o parâmetro` rescale`. Esta classe `ImageDataGenerator` permite que você instancie geradores de lotes de imagens aumentadas (e seus rótulos) via `.flow(data, labels)` ou `.flow_from_directory(directory)`. Esses geradores podem então ser usados ​​com os métodos do modelo Keras que aceitam geradores de dados como entradas: `fit`,` evaluate_generator` e `predict_generator`.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Todas as imagens serão redimensionadas por 1./255
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# Fluxo de imagens de treinamento em lotes de 128 usando o gerador train_datagen
train_generator = train_datagen.flow_from_directory(
        '/tmp/horse-or-human/',  # Este é o diretório de origem para imagens de treinamento
        target_size=(150, 150),  # Todas as imagens serão redimensionadas para 150x150
        batch_size=128,
        # Se usamos a perda binary_crossentropy, precisamos de rótulos binários
        class_mode='binary')

# Flow training images in batches of 128 using train_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        '/tmp/validation-horse-or-human/', # Este é o diretório de origem para imagens de treinamento
        target_size=(150, 150),  # Todas as imagens serão redimensionadas para 150x150
        batch_size=32,
        # Se usamos a perda binary_crossentropy, precisamos de rótulos binários
        class_mode='binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


### Treinamento
Vamos treinar a rede por 15 épocas.

*Loss* e *accuracy*  são ótimos indicadores do progresso do treinamento. Se intenta adivinhar a classificação dos dados de treinamento e, en seguida, compara-los com a etiqueta conhecida, calculando o resultado. O *accuracy* é a parte das suposições corretas.

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8)

Geralmente teremos precisões altas como 1.000, que é provavelmente um sinal de *overfiting*. A precisão do conjunto de validação é de 0.81, o que é muito bom, mas vamos testá-lo com algumas imagens reais.


### Rodando o modelo

Agora vamos dar uma olhada na execução de uma predição usando o modelo. Este código permitirá que você escolha um ou mais arquivos do seu sistema de arquivos, ele os carregará e os executará no modelo, dando uma indicação se o objeto é um cavalo ou um humano.

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # prevendo imagens
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")
 

Saving horse.jpg to horse (1).jpg
[0.]
horse.jpg is a horse


## Limpeza
Para encerrar o kernel e liberar recursos de memória, executar:

In [ ]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)